In [1]:
using DataFrames, CSV, Query, StatsBase

In [2]:
ENV["COLUMNS"] = 1000; ENV["LINES"] = 20;

Loading ISCN Layers dataset

In [3]:
df_ISCN_orig_c1 = CSV.read("ISCN_ALL_DATA_LAYER_C1_1-1.csv", DataFrame)
df_ISCN_orig_c2 = CSV.read("ISCN_ALL_DATA_LAYER_C2_1-1.csv", DataFrame)
df_ISCN_orig_c3 = CSV.read("ISCN_ALL_DATA_LAYER_C3_1-1.csv", DataFrame)
df_ISCN_orig_c4 = CSV.read("ISCN_ALL_DATA_LAYER_C4_1-1.csv", DataFrame)

┌ Warning: thread = 1 warning: only found 93 / 94 columns around data row: 112325. Filling remaining columns with `missing`
└ @ CSV /Users/Daniele/.julia/packages/CSV/la2cd/src/file.jl:603
┌ Warning: thread = 1 warning: only found 93 / 94 columns around data row: 107913. Filling remaining columns with `missing`
└ @ CSV /Users/Daniele/.julia/packages/CSV/la2cd/src/file.jl:603
┌ Warning: thread = 1 warning: only found 93 / 94 columns around data row: 110664. Filling remaining columns with `missing`
└ @ CSV /Users/Daniele/.julia/packages/CSV/la2cd/src/file.jl:603
┌ Warning: thread = 1 warning: only found 93 / 94 columns around data row: 114931. Filling remaining columns with `missing`
└ @ CSV /Users/Daniele/.julia/packages/CSV/la2cd/src/file.jl:603


,dataset_name_sub,dataset_name_soc,lat (dec. deg),long (dec. deg),datum (datum),state (state_province),country (country),site_name,observation_date (YYYY-MM-DD),profile_name,layer_name,layer_top (cm),layer_bot (cm),hzn_desgn_other,hzn,hzn_desgn,layer_note,color,vegclass_local,soil_taxon,soil_series,bd_method,bd_samp (g cm-3),bd_tot (g cm-3),bd_whole (g cm-3),bd_other (g cm-3),bdNRCS_prep_code,cNRCS_prep_code,c_method,c_tot (percent),oc (percent),loi (percent),n_tot (percent),c_to_n (mass ratio),soc (g cm-2),soc_carbon_flag,soc_method,ph_method,ph_cacl,ph_h2o,ph_other,caco3 (percent),sand_tot_psa (percent),silt_tot_psa (percent),clay_tot_psa (percent),wpg2_method,wpg2 (percent),cat_exch (cmol H+ kg-1),al_dith (specified by al_fe_units),al_ox (specified by al_fe_units),al_other (specified by al_fe_units),fe_dith (specified by al_fe_units)
,String,String,String?,String?,String?,String?,String?,String,String?,String,String,Int64?,Int64?,Missing,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,Missing,String?,String?,String?,String?,String?,String?,String?,String?,Int64?,Int64?,Int64?,String?,Int64?,String?,String?,String?,String?,String?
1,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219","-150,686",NAD27,Alaska,United States,S2001AK122001,06/09/2001,01N1159,01N05963,missing,missing,missing,missing,missing,missing,missing,missing,"Medial over loamy, amorphic over mixed, superactive Andic Humicryods",Naptowne,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,27",Eqn 11 + NRCS (c---w),missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
2,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219","-150,686",NAD27,Alaska,United States,S2001AK122001,06/09/2001,01N1159,01N05964,missing,missing,missing,missing,missing,missing,missing,missing,"Medial over loamy, amorphic over mixed, superactive Andic Humicryods",Naptowne,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,27",Eqn 7 + NRCS (c---w),missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
3,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219","-150,686",NAD27,Alaska,United States,S2001AK122001,06/09/2001,01N1159,01N05965,missing,missing,missing,missing,missing,missing,missing,missing,"Medial over loamy, amorphic over mixed, superactive Andic Humicryods",Naptowne,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,12",noFill + NRCS (c-s-w),missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
4,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219","-150,686",NAD27,Alaska,United States,S2001AK122001,06/09/2001,01N1159,01N05966,missing,missing,missing,missing,missing,missing,missing,missing,"Medial over loamy, amorphic over mixed, superactive Andic Humicryods",Naptowne,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,25",noFill + NRCS (c-s-w),missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
5,AK DSC Project SOC stock computation,AK DSC Project SOC stock computation,"60,5219","-150,686",NAD27,Alaska,United States,S2001AK122001,06/09/2001,01N1159,01N05967,missing,missing,missing,missing,missing,missing,missing,missing,"Medial over loamy, amorphic over mixed, superactive Andic Humicryods",Naptowne,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,05",noFill + NRCS (c-s-w),missing,missing,mis

In [4]:
#all names match
all((names(df_ISCN_orig_c1) .== names(df_ISCN_orig_c2)) .& (names(df_ISCN_orig_c3) .== names(df_ISCN_orig_c4)))

true

In [5]:
df1 = deepcopy(df_ISCN_orig_c1)

append!(df1, df_ISCN_orig_c2, promote=true)
append!(df1, df_ISCN_orig_c3, promote=true)
append!(df1, df_ISCN_orig_c4, promote=true)

,dataset_name_sub,dataset_name_soc,lat (dec. deg),long (dec. deg),datum (datum),state (state_province),country (country),site_name,observation_date (YYYY-MM-DD),profile_name,layer_name,layer_top (cm),layer_bot (cm),hzn_desgn_other,hzn,hzn_desgn,layer_note,color,vegclass_local,soil_taxon,soil_series,bd_method,bd_samp (g cm-3),bd_tot (g cm-3),bd_whole (g cm-3),bd_other (g cm-3),bdNRCS_prep_code,cNRCS_prep_code,c_method,c_tot (percent),oc (percent),loi (percent),n_tot (percent),c_to_n (mass ratio),soc (g cm-2),soc_carbon_flag,soc_method,ph_method,ph_cacl,ph_h2o,ph_other,caco3 (percent),sand_tot_psa (percent),silt_tot_psa (percent),clay_tot_psa (percent),wpg2_method,wpg2 (percent),cat_exch (cmol H+ kg-1),al_dith (specified by al_fe_units),al_ox (specified by al_fe_units),al_other (specified by al_fe_units),fe_dith (specified by al_fe_units),fe_ox (specified by al_fe_units)
,String,String,String?,String?,String?,String?,String?,String,String?,String,String,Any,Int64?,Missing,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,Any,String?,Missing,String?,String?,String?,String?,String?,String?,String?,String?,Int64?,Int64?,Int64?,String?,Int64?,String?,String?,String?,String?,String?,String?
1,Heckman lithosequence,Heckman lithosequence,"31,8875","-109,28",WGS84,Arizona,missing,Chiricahua Rhyolite,15/10/2006,CHRY1,CHRY1_A1,missing,missing,missing,missing,missing,missing,missing,missing,"loamy, mixed, superactive, mesic Typic Haplustept",missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"1,12",missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
2,Heckman lithosequence,Heckman lithosequence,"31,8875","-109,28",WGS84,Arizona,missing,Chiricahua Rhyolite,15/10/2006,CHRY1,CHRY1_A2,missing,missing,missing,missing,missing,missing,missing,missing,"loamy, mixed, superactive, mesic Typic Haplustept",missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,63",missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
3,Heckman lithosequence,Heckman lithosequence,"31,8875","-109,28",WGS84,Arizona,missing,Chiricahua Rhyolite,15/10/2006,CHRY1,CHRY1_A3,missing,missing,missing,missing,missing,missing,missing,missing,"loamy, mixed, superactive, mesic Typic Haplustept",missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,14",missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
4,Heckman lithosequence,Heckman lithosequence,"31,8875","-109,28",WGS84,Arizona,missing,Chiricahua Rhyolite,15/10/2006,CHRY1,CHRY1_Bw1,missing,missing,missing,missing,missing,missing,missing,missing,"loamy, mixed, superactive, mesic Typic Haplustept",missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,26",missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
5,Heckman lithosequence,Heckman lithosequence,"31,8875","-109,28",WGS84,Arizona,missing,Chiricahua Rhyolite,15/10/2006,CHRY1,CHRY1_Bw2,missing,missing,missing,missing,missing,missing,missing,missing,"loamy, mixed, superactive, mesic Typic Haplustept",missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,"0,06",missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
6,Heckman lithosequence,Heckman lithosequence,"31,8875","-109,28",WGS84,Arizona,missing,Chiricahua Rhyolite,15/10/2006,CHRY1,CHRY1_BCr,missing,m

In [6]:
function tidy_names(old_names)

return new_names = old_names |>
    #remove leading and trailing spaces
    n -> strip.(n) |>

    #replace spaces with underscores
    n -> replace.(n, ' ' => '_') |> 

    #remove parenthesis
    n -> replace.(n,'(' => "") |> 
    n -> replace.(n,')' => "") |>

    #remove dashes and dots
    n -> replace.(n,'-' => "") |> 
    n -> replace.(n, '.' => "") |>

    #all lowercase
    n -> lowercase.(n)

end

tidy_names (generic function with 1 method)

In [7]:
#copy of merged dataset and tidied up column names
df_ISCN_layers = deepcopy(df1)

rename!(df_ISCN_layers, names(df_ISCN_layers) .=> tidy_names(names(df_ISCN_layers)))

#define new column that has unique key, site_name_profile_name
df_ISCN_layers.sp_name = [(row.site_name * "_" * row.profile_name) for row in eachrow(df_ISCN_layers)]

df_ISCN_layers = df_ISCN_layers |>

#pick relevant columns
@select(:sp_name,:layer_name,:lat_dec_deg,:long_dec_deg,
        :observation_date_yyyymmdd,:hzn,:hzn_desgn,:layer_top_cm,:layer_bot_cm,:oc_percent) |>

#remove layers with missing info
@dropna(:layer_top_cm,:layer_bot_cm,:oc_percent) |>

#parse oc_percent values and keep only non-negative ones
@mutate(oc_percent = replace(_.oc_percent,','=>'.')) |>
@mutate(oc_percent = parse(Float64, _.oc_percent)) |>
@filter(_.oc_percent >= 0) |>

#some layer_top values are wrongly represented as "30/01/1900 0:00" instead of 30
#these values (strings) are longer than 4 chars, keep only first two
@mutate(layer_top_cm = if length(_.layer_top_cm) > 4; _.layer_top_cm[1:2]; else _.layer_top_cm; end) |>


#parsing layer_top_cm to Int and keeping  profiles with depths < 1000cm
#if the value is a string, parse it to Int - otherwise leave it as-is
@mutate(layer_top_cm = isa(_.layer_top_cm, String) ? parse(Int, _.layer_top_cm) : _.layer_top_cm) |>
@filter(_.layer_top_cm < 1000) |>


#keep layers for which top < bot
@filter(_.layer_top_cm < _.layer_bot_cm) |>

#parse latitude values, after dropping missing ones
@dropna(:lat_dec_deg) |>
@mutate(lat_dec_deg = replace(_.lat_dec_deg,','=>'.')) |>
@mutate(lat_dec_deg = parse(Float64, _.lat_dec_deg)) |>

#parse longitude values, after dropping missing ones
@dropna(:long_dec_deg) |>
@mutate(long_dec_deg = replace(_.long_dec_deg,','=>'.')) |>
@mutate(long_dec_deg = parse(Float64, _.long_dec_deg)) |>

@rename(:observation_date_yyyymmdd => :obs_date_yyyymmdd) |> 

#keep only unique layers
@unique() |>


DataFrame


,sp_name,layer_name,lat_dec_deg,long_dec_deg,obs_date_yyyymmdd,hzn,hzn_desgn,layer_top_cm,layer_bot_cm,oc_percent
,String,String,Float64,Float64,String?,String,String?,Int64,Int64,Float64
1,NCC 1_NCC 1,NCC 1.1,35.051,-83.4342,15/07/2009,?,unknown,0,10,3.17
2,NCC 1_NCC 1,NCC 1.2,35.051,-83.4342,15/07/2009,?,unknown,10,30,1.76
3,NCC 1_NCC 1,NCC 1.3,35.051,-83.4342,15/07/2009,?,unknown,30,90,0.89
4,NCC 2_NCC 2,NCC 2.1,35.0484,-83.4342,15/07/2009,?,unknown,0,10,3.91
5,NCC 2_NCC 2,NCC 2.2,35.0484,-83.4342,15/07/2009,?,unknown,10,30,2.34
6,NCC 2_NCC 2,NCC 2.3,35.0484,-83.4342,15/07/2009,?,unknown,30,90,1.7
7,NCC 3_NCC 3,NCC 3.1,35.0473,-83.434,15/07/2009,?,unknown,0,10,2.99
8,NCC 3_NCC 3,NCC 3.2,35.0473,-83.434,15/07/2009,?,unknown,10,30,1.85
9,NCC 3_NCC 3,NCC 3.3,35.0473,-83.434,15/07/2009,?,unknown,30,90,1.22


In [8]:
function continuity_check(profile) 
    #the function assumes layers are ordered (e.g.  0 -> 20 -> 40)

    top = profile.layer_top_cm
    bot = profile.layer_bot_cm

    num_layers = length(top)

    #1st layer is always good
    mask = [true]

    #from 2nd layer downward, we check for matching depths at bot/top
    mask = append!(mask, [top[l] == bot[l-1] for l in 2:num_layers])

    #if we run into a non-matching layer, we mark as 'false' all following ones
    first_noMatch_id = findfirst(!, mask)

    if !isnothing(first_noMatch_id) #true if a non-contiguous layer has been found
        mask[first_noMatch_id:end] .= false
    end

    return mask	
end

continuity_check (generic function with 1 method)

In [9]:
function continuity_check_df(df)
#generate continuity mask for the whole dataframe
layers_keep = Bool[]

for profile in unique(df.sp_name)

    append!(layers_keep,continuity_check(df[df.sp_name.==profile,:]))

end

return layers_keep

end

continuity_check_df (generic function with 1 method)

#### DataFrame with only continuous profiles:

In [10]:
df_ISCN_cont = df_ISCN_layers[continuity_check_df(df_ISCN_layers),:] 

,sp_name,layer_name,lat_dec_deg,long_dec_deg,obs_date_yyyymmdd,hzn,hzn_desgn,layer_top_cm,layer_bot_cm,oc_percent
,String,String,Float64,Float64,String?,String,String?,Int64,Int64,Float64
1,NCC 1_NCC 1,NCC 1.1,35.051,-83.4342,15/07/2009,?,unknown,0,10,3.17
2,NCC 1_NCC 1,NCC 1.2,35.051,-83.4342,15/07/2009,?,unknown,10,30,1.76
3,NCC 1_NCC 1,NCC 1.3,35.051,-83.4342,15/07/2009,?,unknown,30,90,0.89
4,NCC 2_NCC 2,NCC 2.1,35.0484,-83.4342,15/07/2009,?,unknown,0,10,3.91
5,NCC 2_NCC 2,NCC 2.2,35.0484,-83.4342,15/07/2009,?,unknown,10,30,2.34
6,NCC 2_NCC 2,NCC 2.3,35.0484,-83.4342,15/07/2009,?,unknown,30,90,1.7
7,NCC 3_NCC 3,NCC 3.1,35.0473,-83.434,15/07/2009,?,unknown,0,10,2.99
8,NCC 3_NCC 3,NCC 3.2,35.0473,-83.434,15/07/2009,?,unknown,10,30,1.85
9,NCC 3_NCC 3,NCC 3.3,35.0473,-83.434,15/07/2009,?,unknown,30,90,1.22


#### Importing BD values is necessary before inferring layer designation:

In [11]:
df_BD_pred_orig = CSV.read("ISCNTemplate_NRCS_BD_predictions.csv", DataFrame)

┌ Warning: thread = 1 warning: only found 26 / 27 columns around data row: 434583. Filling remaining columns with `missing`
└ @ CSV /Users/Daniele/.julia/packages/CSV/la2cd/src/file.jl:603


,dataset_name,site_name,profile_name,layer_name,layer_top,layer_bot,hzn_desgn,c_method,bd_method,bd_pred_1,bd_pred_2,bd_samp,bd_tot,bd_whole,bd_other,bdNRCS_prep_code,sand_tot_psa,silt_tot_psa,clay_tot_psa,wpg2_method,wpg2,soc,soc_sigma,soc_method,c_tot,oc,cNRCS_prep_code
,String,String,String,String,String,String,String?,String?,String?,String,String,String,String,String,String,String?,String,String,String,String?,String,String,String,String,String,String,String?
1,Boby_Mack,BF-61,BF-61-1,BF61-1-F,0,4,unknown,missing,missing,"1,14","0,73","0,04",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"44,62",NA,missing
2,Boby_Mack,BF-61,BF-61-1,BF61-1-H,4,6,unknown,missing,missing,"1,14","0,74","0,12",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"30,21",NA,missing
3,Boby_Mack,BF-61,BF-61-1,BF61-1-M1,6,11,unknown,missing,missing,"1,27","1,03","1,15",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"5,32",NA,missing
4,Boby_Mack,BF-61,BF-61-1,BF61-1-M2,11,16,unknown,missing,missing,"1,31","1,13","1,12",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"3,88",NA,missing
5,Boby_Mack,BF-61,BF-61-2,BF61-2-DM,0,5,o,missing,missing,"0,56","0,36","0,01",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"45,53",NA,missing
6,Boby_Mack,BF-61,BF-61-2,BF61-2-F,5,19,unknown,missing,missing,"1,34","0,67","0,09",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"48,79",NA,missing
7,Boby_Mack,BF-61,BF-61-2,BF61-2-H,19,22,unknown,missing,missing,"1,29","0,77","0,48",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"19,17",NA,missing
8,Boby_Mack,BF-61,BF-61-2,BF61-2-H:354,22,25,unknown,missing,missing,"1,28","0,67","0,12",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"44,01",NA,missing
9,Boby_Mack,BF-61,BF-61-2,BF61-2-M1,25,30,unknown,missing,missing,"1,31","0,96","1,04",NA,NA,NA,missing,NA,NA,NA,missing,NA,NA,NA,NA,"5,65",NA,missing


In [12]:
#copy of original dataset and tidied up column names
df_BD_pred = deepcopy(df_BD_pred_orig)

#define new column that has unique key, site_name_profile_name
df_BD_pred.sp_name = [(row.site_name * "_" * row.profile_name) for row in eachrow(df_BD_pred)]

df_BD_pred = df_BD_pred |>

#pick relevant columns
@select(:sp_name,:layer_name,:bd_pred_1,:bd_pred_2) |>

##remove layers with missing predicted BD
@dropna(:bd_pred_1, :bd_pred_2) |>

#parse bd_pred_1 values and keep only non-negative ones
@mutate(bd_pred_1 = replace(_.bd_pred_1,','=>'.')) |>
@mutate(bd_pred_1 = parse(Float64, _.bd_pred_1)) |>
@filter(_.bd_pred_1 > 0) |>

#same with bd_pred_2 values
@mutate(bd_pred_2 = replace(_.bd_pred_2,','=>'.')) |>
@mutate(bd_pred_2 = parse(Float64, _.bd_pred_2)) |>
@filter(_.bd_pred_2 > 0) |>

#keep only unique sites/profiles/layers
@unique((_.sp_name, _.layer_name)) |>

#‘bd_pred_1’ to be used for organic horizons, ‘be_pred_2’ for mineral soil horizons
@rename(:bd_pred_1 => :bd_organic) |>
@rename(:bd_pred_2 => :bd_mineral) |>

DataFrame

,sp_name,layer_name,bd_organic,bd_mineral
,String,String,Float64,Float64
1,BF-61_BF-61-1,BF61-1-F,1.14,0.73
2,BF-61_BF-61-1,BF61-1-H,1.14,0.74
3,BF-61_BF-61-1,BF61-1-M1,1.27,1.03
4,BF-61_BF-61-1,BF61-1-M2,1.31,1.13
5,BF-61_BF-61-2,BF61-2-DM,0.56,0.36
6,BF-61_BF-61-2,BF61-2-F,1.34,0.67
7,BF-61_BF-61-2,BF61-2-H,1.29,0.77
8,BF-61_BF-61-2,BF61-2-H:354,1.28,0.67
9,BF-61_BF-61-2,BF61-2-M1,1.31,0.96


In [13]:
function infer_designation(old_desig, blk)
#takes layer and returns its inferred soil designation, either 'organic' or 'mineral'

#old_desig = get(existing_desig)
new_desig = "no_desig" #value to be eventually returned

#blk = get(bulk_density) #already parsed to F64, no missing values assumed
threshold = 0.5

control_desig = ['O','o','H','h'] #possible chars in existing desig that indicate presence of organic material in the layer

new_desig_organic = "organic"
new_desig_mineral = "mineral"

if old_desig == ""
    #check on bulk density value to decide which designation applies 
    new_desig = (blk <= threshold) ? new_desig_organic : new_desig_mineral

#designation not missing			
else	
    check_desig = [in(c,control_desig) for c in old_desig] #check if any char in the existing designation matches organic control
    check_desig_BD = !isnothing(findfirst(check_desig)) & (blk <= threshold) #true if any char matches and BD < treshold

    new_desig = check_desig_BD ? new_desig_organic : new_desig_mineral
end


return new_desig
end

infer_designation (generic function with 1 method)

In [14]:
#adding predicted BD values to each layer
df_hz = innerjoin(df_ISCN_cont, df_BD_pred, on = [:sp_name, :layer_name])	
unique!(df_hz, [:sp_name, :layer_name])
	
#work on layer designation
df_hz.hzn = replace(df_hz.hzn, "?" => "") 
df_hz.hzn = replace(df_hz.hzn, missing => "")
	
df_hz.hzn_desgn = replace(df_hz.hzn_desgn, "unknown" => "") 
df_hz.hzn_desgn = replace(df_hz.hzn_desgn, missing => "") 

#create new column with concatenated designations, but leave row empty if both are empty
df_hz.hzn_merged = [!isempty(row.hzn * row.hzn_desgn) ? (row.hzn * "_" * row.hzn_desgn) : "" for row in eachrow(df_hz)]
	
#infer layer designation and write to new col, using only bd_organic for threshold check
df_hz.infer_DESIG = [infer_designation(r.hzn_merged,r.bd_organic) for r in eachrow(df_hz) ]	

#getting rid of old designation cols
select!(df_hz, Not(:hzn))	
select!(df_hz, Not(:hzn_desgn))	


,sp_name,layer_name,lat_dec_deg,long_dec_deg,obs_date_yyyymmdd,layer_top_cm,layer_bot_cm,oc_percent,bd_organic,bd_mineral,hzn_merged,infer_DESIG
,String,String,Float64,Float64,String?,Int64,Int64,Float64,Float64,Float64,String,String
1,NCC 1_NCC 1,NCC 1.1,35.051,-83.4342,15/07/2009,0,10,3.17,1.24,1.39,,mineral
2,NCC 1_NCC 1,NCC 1.2,35.051,-83.4342,15/07/2009,10,30,1.76,1.33,1.42,,mineral
3,NCC 1_NCC 1,NCC 1.3,35.051,-83.4342,15/07/2009,30,90,0.89,1.33,1.35,,mineral
4,NCC 2_NCC 2,NCC 2.1,35.0484,-83.4342,15/07/2009,0,10,3.91,1.24,1.39,,mineral
5,NCC 2_NCC 2,NCC 2.2,35.0484,-83.4342,15/07/2009,10,30,2.34,1.33,1.42,,mineral
6,NCC 2_NCC 2,NCC 2.3,35.0484,-83.4342,15/07/2009,30,90,1.7,1.33,1.35,,mineral
7,NCC 3_NCC 3,NCC 3.1,35.0473,-83.434,15/07/2009,0,10,2.99,1.24,1.39,,mineral
8,NCC 3_NCC 3,NCC 3.2,35.0473,-83.434,15/07/2009,10,30,1.85,1.33,1.42,,mineral
9,NCC 3_NCC 3,NCC 3.3,35.0473,-83.434,15/07/2009,30,90,1.22,1.33,1.35,,mineral


In [15]:
function depth_clip(profile; depth_min=0, depth_max=100)
#takes in a profile with N layers and returns a clipped profile, by default from 0 to 100cm
#run this function ALWAYS AFTER checking for layers continuity


top = profile.layer_top_cm
bot = deepcopy(profile.layer_bot_cm) #create copy of the original profile bot, to be edited later

n_layers = length(top)

#init array of 'true' -> keep all layers unless proven guilty
layers_keep = Vector{Bool}(undef,n_layers)
layers_keep .= true

#if profile doesn't reach threshold, or starts after min, ditch the whole profile
if maximum(bot) < depth_max || minimum(top) > depth_min; 	

    layers_keep .= false

#othewise mark layers to drop above threshold, and clip the overlapping one
else 
    for lr in 1:n_layers
        #simple case, both top and bot below threshold
        if top[lr] >= depth_max; layers_keep[lr] = false; end

        #layers overlapping threshold
        if top[lr] < depth_max && bot[lr] > depth_max
            bot[lr] = depth_max
        end

    end

    profile.layer_bot_cm = bot

end


return profile[layers_keep,:]

end

depth_clip (generic function with 1 method)

In [16]:
function compute_clipped_profiles(df)
#initialize empty DF with same columns and types as starting one
results = DataFrame(eltypes(df), names(df))

for profile in unique(df.sp_name)

	append!(results, depth_clip(df[df.sp_name .== profile,:]))

end

return results

end

compute_clipped_profiles (generic function with 1 method)

In [17]:
df_hz_clipped = compute_clipped_profiles(df_hz)

,sp_name,layer_name,lat_dec_deg,long_dec_deg,obs_date_yyyymmdd,layer_top_cm,layer_bot_cm,oc_percent,bd_organic,bd_mineral,hzn_merged,infer_DESIG
,String,String,Float64,Float64,String?,Int64,Int64,Float64,Float64,Float64,String,String
1,NMJ 4_NMJ 4,NMJ 4.1,32.5738,-106.615,15/01/2010,0,2,0.5,0.9,1.25,A_A,mineral
2,NMJ 4_NMJ 4,NMJ 4.2,32.5738,-106.615,15/01/2010,2,60,1.03,1.28,1.33,B_BW,mineral
3,NMJ 4_NMJ 4,NMJ 4.3,32.5738,-106.615,15/01/2010,60,100,2.11,1.37,1.34,B_BK,mineral
4,04KY-007-001_04KY-007-001,KY0099,37.1613,-89.0593,15/04/2004,0,18,2.15,1.33,1.42,A_Ap,mineral
5,04KY-007-001_04KY-007-001,KY0100,37.1613,-89.0593,15/04/2004,18,28,0.84,1.41,1.39,B_Bt1,mineral
6,04KY-007-001_04KY-007-001,KY0101,37.1613,-89.0593,15/04/2004,28,58,0.52,1.45,1.38,B_Bt2,mineral
7,04KY-007-001_04KY-007-001,KY0102,37.1613,-89.0593,15/04/2004,58,84,0.34,1.46,1.37,B_Bt3,mineral
8,04KY-007-001_04KY-007-001,KY0103,37.1613,-89.0593,15/04/2004,84,100,0.25,1.48,1.35,B_Bt4,mineral
9,03KY145003_04KY-145-001,KY0136,37.1135,-88.7034,31/10/2003,0,23,2.3,1.36,1.44,A_Ap,mineral


#### For the ISCN dataset, organic carbon stock can be calculated as:

( 100cm x 100cm x layer\_height [cm] x bulk\_density [g/cm³] x oc_percent / 100 ) / 1000 

ORG carbon stock in [Kg]

In [18]:
function orgc_storage_desig(profile) 

id = profile.sp_name[1]
top = profile.layer_top_cm
bot = profile.layer_bot_cm
des = profile.infer_DESIG

org_bd = profile.bd_organic #[g/cm³]
min_bd = profile.bd_mineral

orgc = profile.oc_percent #[%]

layer_height = (bot.-top)
num_layers = length(top)

orgc_stored_mineral = 0.0
orgc_stored_organic = 0.0


for l in 1:num_layers
    #using different BD values depending on layer designation
    if des[l] == "organic"; orgc_stored_organic += ( 100 * 100 * layer_height[l] ) * ( org_bd[l] * orgc[l] / 100 ) / 1000; end
    if des[l] == "mineral"; orgc_stored_mineral += ( 100 * 100 * layer_height[l] ) * ( min_bd[l] * orgc[l] / 100 ) / 1000; end

end

return DataFrame(profile_id = id, 
                 orgc_stored_mineral = orgc_stored_mineral, 
                 orgc_stored_organic = orgc_stored_organic)
end

orgc_storage_desig (generic function with 1 method)

In [19]:
function compute_orgc_storage_desig(df)

    unique_profiles = unique(df.sp_name)

    #initialize result var with 1st call to function
    result = orgc_storage_desig(df[df.sp_name .== unique_profiles[1],:])

    for profile in unique_profiles[2:end]

        append!(result, orgc_storage_desig(df[df.sp_name .== profile,:]))

    end

    return result
end

compute_orgc_storage_desig (generic function with 1 method)

In [20]:
#compute stored ORGC for continuous, not clipped, profiles
df_storage_cont = compute_orgc_storage_desig(df_hz)

,profile_id,orgc_stored_mineral,orgc_stored_organic
,String,Float64,Float64
1,NCC 1_NCC 1,16.6137,0.0
2,NCC 2_NCC 2,25.8505,0.0
3,NCC 3_NCC 3,19.2921,0.0
4,NCC 4_NCC 4,16.987,0.0
5,NCC 5_NCC 5,38.908,0.0
6,NMJ 1_NMJ 1,38.9916,0.0
7,NMJ 2_NMJ 2,31.2134,0.0
8,NMJ 3_NMJ 3,24.7325,0.0
9,NMJ 4_NMJ 4,19.38,0.0


In [21]:
#compute stored ORGC for continuous and clipped profiles
df_storage_clipped = compute_orgc_storage_desig(df_hz_clipped);

In [22]:
#profiles' info from the original dataset
df_info = df_hz[:,[:sp_name, :lat_dec_deg, :long_dec_deg, :obs_date_yyyymmdd]]
rename!(df_info, :sp_name => :profile_id )
unique!(df_info)

,profile_id,lat_dec_deg,long_dec_deg,obs_date_yyyymmdd
,String,Float64,Float64,String?
1,NCC 1_NCC 1,35.051,-83.4342,15/07/2009
2,NCC 2_NCC 2,35.0484,-83.4342,15/07/2009
3,NCC 3_NCC 3,35.0473,-83.434,15/07/2009
4,NCC 4_NCC 4,35.0385,-83.4598,15/07/2009
5,NCC 5_NCC 5,35.031,-83.4619,15/07/2009
6,NMJ 1_NMJ 1,32.5603,-106.79,15/01/2010
7,NMJ 2_NMJ 2,32.5734,-106.672,15/01/2010
8,NMJ 3_NMJ 3,32.5717,-106.644,15/01/2010
9,NMJ 4_NMJ 4,32.5738,-106.615,15/01/2010


In [23]:
function depth_desig(profile) 

    id = profile.sp_name[1]
    des = profile.infer_DESIG
    top = profile.layer_top_cm
    bot = profile.layer_bot_cm

    missing_val = -1

    num_layers = length(top)

    mineral_index = [i for i in 1:num_layers if des[i] == "mineral"]
    organic_index = [i for i in 1:num_layers if des[i] == "organic"]

    #values to return

    profile_top = minimum(top)
    profile_bot = maximum(bot)

    mineral_top = isempty(mineral_index) ? missing_val : minimum(top[mineral_index])
    mineral_bot = isempty(mineral_index) ? missing_val : maximum(bot[mineral_index])

    organic_top = isempty(organic_index) ? missing_val : minimum(top[organic_index])
    organic_bot = isempty(organic_index) ? missing_val : maximum(bot[organic_index])


    return DataFrame(profile_id = id, 
                     profile_top = profile_top,
                     profile_bot = profile_bot,
                     organic_top = organic_top,
                     organic_bot = organic_bot,
                     mineral_top = mineral_top,
                     mineral_bot = mineral_bot)


end

depth_desig (generic function with 1 method)

In [24]:
function compute_depth_desig(df)

    unique_profiles = unique(df.sp_name)

    #initialize results with 1st call to function
    result = depth_desig(df[df.sp_name .== unique_profiles[1],:])

    for profile in unique_profiles[2:end]

        append!(result, depth_desig(df[df.sp_name .== profile,:]))

    end

    return result
end

compute_depth_desig (generic function with 1 method)

In [25]:
#designations depths per profile (non clipped)
df_cont_depth_desig = compute_depth_desig(df_hz)

,profile_id,profile_top,profile_bot,organic_top,organic_bot,mineral_top,mineral_bot
,String,Int64,Int64,Int64,Int64,Int64,Int64
1,NCC 1_NCC 1,0,90,-1,-1,0,90
2,NCC 2_NCC 2,0,90,-1,-1,0,90
3,NCC 3_NCC 3,0,90,-1,-1,0,90
4,NCC 4_NCC 4,0,90,-1,-1,0,90
5,NCC 5_NCC 5,0,90,-1,-1,0,90
6,NMJ 1_NMJ 1,0,50,-1,-1,0,50
7,NMJ 2_NMJ 2,0,60,-1,-1,0,60
8,NMJ 3_NMJ 3,0,45,-1,-1,0,45
9,NMJ 4_NMJ 4,0,100,-1,-1,0,100


In [26]:
#designations depths per clipped profiles 
df_clipped_depth_desig = compute_depth_desig(df_hz_clipped);

In [27]:
#joining results with extra info	
df_cont_results = innerjoin(df_storage_cont, df_cont_depth_desig, df_info, on = :profile_id)

,profile_id,orgc_stored_mineral,orgc_stored_organic,profile_top,profile_bot,organic_top,organic_bot,mineral_top,mineral_bot,lat_dec_deg,long_dec_deg,obs_date_yyyymmdd
,String,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,String?
1,NCC 1_NCC 1,16.6137,0.0,0,90,-1,-1,0,90,35.051,-83.4342,15/07/2009
2,NCC 2_NCC 2,25.8505,0.0,0,90,-1,-1,0,90,35.0484,-83.4342,15/07/2009
3,NCC 3_NCC 3,19.2921,0.0,0,90,-1,-1,0,90,35.0473,-83.434,15/07/2009
4,NCC 4_NCC 4,16.987,0.0,0,90,-1,-1,0,90,35.0385,-83.4598,15/07/2009
5,NCC 5_NCC 5,38.908,0.0,0,90,-1,-1,0,90,35.031,-83.4619,15/07/2009
6,NMJ 1_NMJ 1,38.9916,0.0,0,50,-1,-1,0,50,32.5603,-106.79,15/01/2010
7,NMJ 2_NMJ 2,31.2134,0.0,0,60,-1,-1,0,60,32.5734,-106.672,15/01/2010
8,NMJ 3_NMJ 3,24.7325,0.0,0,45,-1,-1,0,45,32.5717,-106.644,15/01/2010
9,NMJ 4_NMJ 4,19.38,0.0,0,100,-1,-1,0,100,32.5738,-106.615,15/01/2010


In [28]:
df_clipped_results = innerjoin(df_storage_clipped, df_clipped_depth_desig, df_info, on = :profile_id)

,profile_id,orgc_stored_mineral,orgc_stored_organic,profile_top,profile_bot,organic_top,organic_bot,mineral_top,mineral_bot,lat_dec_deg,long_dec_deg,obs_date_yyyymmdd
,String,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,String?
1,NMJ 4_NMJ 4,19.38,0.0,0,100,-1,-1,0,100,32.5738,-106.615,15/01/2010
2,04KY-007-001_04KY-007-001,10.5669,0.0,0,100,-1,-1,0,100,37.1613,-89.0593,15/04/2004
3,03KY145003_04KY-145-001,12.6387,0.0,0,100,-1,-1,0,100,37.1135,-88.7034,31/10/2003
4,04KY-145-002_04KY-145-002,9.60141,0.0,0,100,-1,-1,0,100,37.1123,-88.7036,31/10/2003
5,04KY145001_04KY-145-003,9.18743,0.0,0,100,-1,-1,0,100,37.1093,-88.6962,04/11/2003
6,04KY145004_04KY-145-004,7.3845,0.0,0,100,-1,-1,0,100,36.9536,-88.515,05/08/2003
7,67FN220000_40A0165,18.3729,0.0,0,100,-1,-1,0,100,-15.6667,-47.7833,01/03/1967
8,S1959PR019008_40A0166,15.225,0.0,0,100,-1,-1,0,100,18.2392,-66.2839,16/12/1959
9,S1961TX389003_40A0171,6.18199,0.0,0,100,-1,-1,0,100,31.3936,-103.518,26/10/1961


Results here:

(full) https://drive.google.com/file/d/1v8YxVTfFq0x4MhkRtBLtFGgBQIC_9RFI/view?usp=sharing

(clipped) https://drive.google.com/file/d/1Gw8uGphn3Mup8OPABXaumtgQEg37UIj6/view?usp=sharing
